In [3]:
from IPython import display
import ultralytics
from ultralytics import YOLO
import cv2
import numpy as np

display.clear_output()
ultralytics.checks()

Ultralytics YOLOv8.0.200  Python-3.11.3 torch-2.1.0+cpu CPU (Intel Core(TM) i7-7700 3.60GHz)
Setup complete  (8 CPUs, 16.0 GB RAM, 141.9/223.6 GB disk)


In [4]:
model = YOLO(model="D:/dip/yolov8/runs/detect/train/weights/best.pt")
img = cv2.imread('./dataset/images/c6ebf6ea-resized_600x600_005fa4a45c75b075.jpg')
result = model.predict(source=img, conf=0.25)


0: 640x640 9 Dogs, 113.7ms
Speed: 7.0ms preprocess, 113.7ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)


In [5]:
mask = result[0].masks

In [2]:
mask = result[0].masks
overlay = img.copy()
mbox = result[0]

for box in mbox.boxes:
    class_id = mbox.names[box.cls[0].item()]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    conf = round(box.conf[0].item(),2)
    display = f'{class_id} {conf}%'

    image = cv2.putText(img, display, (cords[0], cords[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
    image = cv2.rectangle(image, (cords[0],cords[1]), (cords[2], cords[3]), (255,255,0), 2)

    if(mask!=None):
        for i in range(len(result[0].mask.xy)):
            x = mask[i].astype(int)
            pts = x.reshape((-1, 1, 2))
            image = cv2.polylines(image, [pts], isClosed=True,color=(0,0,255), thickness=2)
            overlay = cv2.fillPoly(overlay, pts=[pts], color=(0,0,255))
        alpha = 0.4
        image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)
    
    r = 600.0 / image.shape[1]
    dim = (600, int(img.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    cv2.imshow('Marked Image', resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


NameError: name 'result' is not defined